In [95]:
from IPython.display import Image

In [117]:
import numpy as np
import pandas as pd
#from keras.optimizers import SGD
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from keras.datasets import cifar10

In [118]:
from tensorflow.keras.optimizers import SGD

I'll be using the Cifra10 dataset 

In [119]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [35]:
# Declare variables
batch_size = 32  #  smaller batch size means more updates in one epoch
num_classes = 10 # number of cifar-10 dataset classes
epochs = 5 # repeat 5 times

- The pixel values are in the range of 0 to 255 for each of the red, green and blue channels.
- Because the input values are well understood, we can easily normalize to the range 0 to 1 by dividing each value by the maximum observation which is 255.
- The data is loaded as integers, so we must cast it to float point values in order to perform the division.

In [120]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train  /= 255
X_test /= 255

CNN architecture 1 :
- We will use a model with 2 convolutional layers followed by max pooling and a flattening out of the network to fully connected layers to make predictions : 
    - Convolutional input layer, 32 feature maps with a size of 3×3, a rectifier activation function
    - Max Pool layer with size 2×2
    - Convolutional input layer, 64 feature maps with a size of 3×3, a rectifier activation function
    - Max Pool layer with size 2×2
    - Flatten layer
    - Fully connected layer with 64 units and a rectifier activation function
    - Fully connected output layer with 10 units and a softmax activation function

In [121]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Adding the fully connected layers to CNN
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
#model summary
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 2304)              0         
                                                                 
 dense_14 (Dense)            (None, 64)               

In [122]:
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/5
1563/1563 [==============================] - 29s 17ms/step - loss: 1.5016 - accuracy: 0.4616
Epoch 2/5
1563/1563 [==============================] - 31s 20ms/step - loss: 1.1228 - accuracy: 0.6077
Epoch 3/5
1563/1563 [==============================] - 31s 20ms/step - loss: 0.9813 - accuracy: 0.6577
Epoch 4/5
1563/1563 [==============================] - 30s 19ms/step - loss: 0.8848 - accuracy: 0.6931
Epoch 5/5
1563/1563 [==============================] - 35s 22ms/step - loss: 0.8080 - accuracy: 0.7202


In [46]:
y_pred = model.predict(X_test)

In [53]:
from sklearn import metrics
#accuracy on training set
y_pred1_train = model.predict(X_train)
print("Training set accuracy:", metrics.accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred1_train, axis=1)))

Training set accuracy: 0.7309


In [55]:
#accuracy on test set 
print("Test set accuracy:", metrics.accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Test set accuracy: 0.6728


10-fold cross-validation on the training set :

In [123]:
from sklearn.model_selection import train_test_split
n_folds = 10
cv_scores2 = list()
for _ in range(n_folds):
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    # evaluate model
    val_acc = model.evaluate(X_train, y_train)
    cv_scores2.append(val_acc)
    
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores2), np.std(cv_scores2)))

1563/1563 [==============================] - 13s 8ms/step - loss: 39.5317 - accuracy: 0.1099
Estimated Accuracy 19.821 (19.711)


It was difficult to do the kfold cross validation since the dataset used wasn't a csv file that I could use train_test_split with easily, which why it just ended up being the same split used for every fold.

Confusion matrix

In [59]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

array([[771,  32,  50,   7,  28,  15,  16,  12,  14,  55],
       [ 22, 839,   4,   6,   3,   8,  11,   3,   8,  96],
       [ 67,   9, 407,  45, 184, 123, 102,  35,   6,  22],
       [ 25,  23,  25, 389,  94, 274,  99,  40,   7,  24],
       [ 21,   2,  26,  44, 704,  37,  70,  76,   6,  14],
       [ 14,   5,  23, 102,  57, 680,  42,  51,   5,  21],
       [  5,  10,  23,  28,  45,  33, 834,   4,   2,  16],
       [ 19,   2,  13,  19,  94,  83,   9, 726,   3,  32],
       [155,  93,  11,  13,  15,  19,  17,  17, 568,  92],
       [ 27, 102,   5,   8,   7,  14,   7,  16,   4, 810]], dtype=int64)

In [62]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.68      0.77      0.73      1000
           1       0.75      0.84      0.79      1000
           2       0.69      0.41      0.51      1000
           3       0.59      0.39      0.47      1000
           4       0.57      0.70      0.63      1000
           5       0.53      0.68      0.59      1000
           6       0.69      0.83      0.76      1000
           7       0.74      0.73      0.73      1000
           8       0.91      0.57      0.70      1000
           9       0.69      0.81      0.74      1000

    accuracy                           0.67     10000
   macro avg       0.68      0.67      0.67     10000
weighted avg       0.68      0.67      0.67     10000



In [91]:
from sklearn.metrics import precision_score
print(f"Precision Score of the classifier is: {precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average ='weighted')}")
#recall
from sklearn.metrics import recall_score
print(f"Recall Score of the classifier is: {recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1),average ='weighted')}")

Precision Score of the classifier is: 0.6847148585151126
Recall Score of the classifier is: 0.6728


CNN architecture 2
- We will use a model with 1 convolutional layer followed by max pooling and a flattening out of the network to fully connected layers to make predictions : 
    - Convolutional input layer, 32 feature maps with a size of 3×3, a rectifier activation function
    - Max Pool layer with size 2×2
    - Flatten layer
    - Fully connected layer with 64 units and a rectifier activation function
    - Fully connected output layer with 10 units and a softmax activation function

In [74]:
def model2():

    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:])) 
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    return model    

In [76]:
cnn2 = model2()
cnn2.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)
y_pred4_train = cnn2.predict(X_train)
print("Training set accuracy:", metrics.accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred4_train, axis=1)))
y_pred4 = cnn2.predict(X_test)
print("Test set accuracy:", metrics.accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred4, axis=1)))

Epoch 1/5
1563/1563 [==============================] - 24s 15ms/step - loss: 1.5272 - accuracy: 0.4568
Epoch 2/5
1563/1563 [==============================] - 26s 16ms/step - loss: 1.1951 - accuracy: 0.5800
Epoch 3/5
1563/1563 [==============================] - 26s 17ms/step - loss: 1.0760 - accuracy: 0.6246
Epoch 4/5
1563/1563 [==============================] - 26s 17ms/step - loss: 0.9985 - accuracy: 0.6540
Epoch 5/5
1563/1563 [==============================] - 25s 16ms/step - loss: 0.9376 - accuracy: 0.6742
Training set accuracy: 0.7055
Test set accuracy: 0.635


In [82]:
confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred4, axis=1))

array([[736,  22,  48,  18,  23,   4,   8,   6,  98,  37],
       [ 46, 742,   3,  13,   7,   5,   4,   5,  56, 119],
       [102,  13, 361,  71, 257,  72,  42,  45,  25,  12],
       [ 43,  10,  48, 449, 171, 145,  52,  33,  29,  20],
       [ 33,   6,  39,  55, 759,  25,  35,  33,  12,   3],
       [ 25,   6,  49, 174, 129, 514,  15,  61,  18,   9],
       [ 17,   5,  28,  95, 166,  22, 630,  10,  12,  15],
       [ 39,   5,  26,  52, 115,  73,   2, 665,   4,  19],
       [101,  49,   6,  12,   7,  10,   5,   4, 770,  36],
       [ 64,  99,  12,  18,  11,  10,   5,  15,  42, 724]], dtype=int64)

In [81]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred4, axis=1)))

              precision    recall  f1-score   support

           0       0.61      0.74      0.67      1000
           1       0.78      0.74      0.76      1000
           2       0.58      0.36      0.45      1000
           3       0.47      0.45      0.46      1000
           4       0.46      0.76      0.57      1000
           5       0.58      0.51      0.55      1000
           6       0.79      0.63      0.70      1000
           7       0.76      0.67      0.71      1000
           8       0.72      0.77      0.75      1000
           9       0.73      0.72      0.73      1000

    accuracy                           0.64     10000
   macro avg       0.65      0.64      0.63     10000
weighted avg       0.65      0.64      0.63     10000



In [92]:
print(f"Precision Score of the classifier is: {precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred4, axis=1), average ='weighted')}")
print(f"Recall Score of the classifier is: {recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred4, axis=1),average ='weighted')}")

Precision Score of the classifier is: 0.6480980358469043
Recall Score of the classifier is: 0.635


CNN architecture 3
- We will use a model with 3 convolutional layers followed by max pooling and a flattening out of the network to fully connected layers to make predictions : 
    - Convolutional input layer, 32 feature maps with a size of 3×3, a rectifier activation function
    - Max Pool layer with size 2×2
    - Convolutional input layer, 64 feature maps with a size of 3×3, a rectifier activation function
    - Max Pool layer with size 2×2
    - Convolutional input layer, 64 feature maps with a size of 3×3, a rectifier activation function
    - Max Pool layer with size 2×2
    - Flatten layer
    - Fully connected layer with 64 units and a rectifier activation function
    - Fully connected output layer with 10 units and a softmax activation function

In [77]:
def model3():

    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:])) 
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    return model    

In [78]:
cnn3 = model3()
cnn3.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)
y_pred3_train = cnn3.predict(X_train)
print("Training set accuracy:", metrics.accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred3_train, axis=1)))
y_pred3 = cnn3.predict(X_test)
print("Test set accuracy:", metrics.accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred3, axis=1)))

Epoch 1/5
1563/1563 [==============================] - 38s 24ms/step - loss: 1.5786 - accuracy: 0.4275
Epoch 2/5
1563/1563 [==============================] - 41s 26ms/step - loss: 1.1979 - accuracy: 0.5756
Epoch 3/5
1563/1563 [==============================] - 41s 26ms/step - loss: 1.0262 - accuracy: 0.6416
Epoch 4/5
1563/1563 [==============================] - 39s 25ms/step - loss: 0.9161 - accuracy: 0.6819
Epoch 5/5
1563/1563 [==============================] - 43s 27ms/step - loss: 0.8354 - accuracy: 0.7093
Training set accuracy: 0.75696
Test set accuracy: 0.7051


In [83]:
confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred3, axis=1))

array([[745,  26,  60,  12,   6,   1,   9,  18,  82,  41],
       [ 14, 839,  11,   5,   1,   2,  10,   9,  27,  82],
       [ 67,   6, 654,  60,  46,  38,  65,  44,  12,   8],
       [ 18,  13, 118, 575,  50,  82,  58,  49,  15,  22],
       [ 28,   2, 102,  71, 571,  26,  61, 118,  14,   7],
       [ 17,   9, 110, 214,  40, 483,  31,  78,   5,  13],
       [  3,   9,  72,  57,  22,   8, 813,   9,   4,   3],
       [ 15,   2,  47,  47,  36,  32,  10, 792,   4,  15],
       [ 76,  53,  21,  15,   2,   3,   4,   8, 782,  36],
       [ 24,  75,  19,  18,   5,   3,   7,  21,  31, 797]], dtype=int64)

In [84]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred3, axis=1)))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74      1000
           1       0.81      0.84      0.82      1000
           2       0.54      0.65      0.59      1000
           3       0.54      0.57      0.55      1000
           4       0.73      0.57      0.64      1000
           5       0.71      0.48      0.58      1000
           6       0.76      0.81      0.79      1000
           7       0.69      0.79      0.74      1000
           8       0.80      0.78      0.79      1000
           9       0.78      0.80      0.79      1000

    accuracy                           0.71     10000
   macro avg       0.71      0.71      0.70     10000
weighted avg       0.71      0.71      0.70     10000



In [93]:
print(f"Precision Score of the classifier is: {precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred3, axis=1), average ='weighted')}")
print(f"Recall Score of the classifier is: {recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred3, axis=1),average ='weighted')}")

Precision Score of the classifier is: 0.7102595632577017
Recall Score of the classifier is: 0.7051


CNN Architecture 4 
- same as architecture 3
- changed epoches to 10 instead of 5 when fitting the model

In [79]:
cnn3.fit(X_train, y_train, batch_size=batch_size, epochs=10)
y_pred5_train = cnn3.predict(X_train)
print("Training set accuracy:", metrics.accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred5_train, axis=1)))
y_pred5 = cnn3.predict(X_test)
print("Test set accuracy:", metrics.accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred5, axis=1)))

Epoch 1/10
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7726 - accuracy: 0.7314
Epoch 2/10
1563/1563 [==============================] - 40s 25ms/step - loss: 0.7238 - accuracy: 0.7474
Epoch 3/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.6825 - accuracy: 0.7613
Epoch 4/10
1563/1563 [==============================] - 40s 26ms/step - loss: 0.6407 - accuracy: 0.7774
Epoch 5/10
1563/1563 [==============================] - 41s 26ms/step - loss: 0.6114 - accuracy: 0.7891
Epoch 6/10
1563/1563 [==============================] - 41s 26ms/step - loss: 0.5835 - accuracy: 0.7989
Epoch 7/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.5567 - accuracy: 0.8060
Epoch 8/10
1563/1563 [==============================] - 41s 26ms/step - loss: 0.5337 - accuracy: 0.8149
Epoch 9/10
1563/1563 [==============================] - 42s 27ms/step - loss: 0.5072 - accuracy: 0.8252
Epoch 10/10
1563/1563 [==============================] - 40s 26m

In [85]:
confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred5, axis=1))

array([[777,  10,  36,  10,  12,   3,   8,   8,  63,  73],
       [ 22, 734,   6,   7,   4,   4,  11,   5,  33, 174],
       [ 75,   2, 554,  54,  92,  77,  78,  20,  17,  31],
       [ 29,   2,  47, 500,  60, 172,  75,  46,  31,  38],
       [ 28,   0,  41,  58, 699,  35,  50,  64,  10,  15],
       [ 21,   2,  45, 116,  52, 628,  41,  55,  10,  30],
       [ 12,   1,  23,  41,  24,  18, 849,   7,   6,  19],
       [ 24,   2,  23,  23,  57,  44,  14, 765,   6,  42],
       [ 84,  15,   4,  10,   5,   3,  10,   7, 808,  54],
       [ 26,  24,   8,   6,   0,   8,   5,   9,  16, 898]], dtype=int64)

In [86]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred5, axis=1)))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74      1000
           1       0.93      0.73      0.82      1000
           2       0.70      0.55      0.62      1000
           3       0.61      0.50      0.55      1000
           4       0.70      0.70      0.70      1000
           5       0.63      0.63      0.63      1000
           6       0.74      0.85      0.79      1000
           7       0.78      0.77      0.77      1000
           8       0.81      0.81      0.81      1000
           9       0.65      0.90      0.76      1000

    accuracy                           0.72     10000
   macro avg       0.73      0.72      0.72     10000
weighted avg       0.73      0.72      0.72     10000



In [94]:
print(f"Precision Score of the classifier is: {precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred5, axis=1), average ='weighted')}")
print(f"Recall Score of the classifier is: {recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred5, axis=1),average ='weighted')}")

Precision Score of the classifier is: 0.7254516904808731
Recall Score of the classifier is: 0.7212


 metrics summary table : 

In [97]:
Image(url="10.PNG", width=400, height=400)

- Architecture 4 performed best overall, it had the most accurate predictions and a balanced precision and recall.
- Architecture 4 had similar number of convolution layers and overall structure as architecture 3, which explains the close results however architecture 4 had double the number of repetitions which resulted in improved results.
- Overall, the less convolution layers we used the less accurate the model was.
- A higher number of epoches imporves the results as well but we have to be careful with this paramter since too many epoches can lead to overfitting if there isn't enough data and early stopping is a approach thar can tell us appropriate epochs without overfitting the model